In [1]:
#_*_coding: utf-8_*_
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import DOUBLE_PRECISION
import sqlalchemy
import psycopg2
import time
import os

In [2]:
conn_string = os.environ['PG_DB']
cur_path = os.getcwd()
ser = Service(cur_path+'/chromedriver')

In [3]:
df_k = pd.read_excel("keywords.xlsx", sheet_name=None)
list_word=[]
for project in df_k.keys():
    #display(df_k[project]['Статистика по словам'][0])
    list_word.append(df_k[project]['Статистика по словам'][0])
#display(list_word)
print(len(list_word))

1


In [4]:
options = webdriver.ChromeOptions()
# get source code
df_list=[]
for keyword in list_word:
    options.add_argument("--user-data-dir="+cur_path+"/profile")
    browser=webdriver.Chrome(service=ser, options=options)
    browser.get(r'https://wordstat.yandex.ru/#!/history?period=weekly&words='+keyword)
    time.sleep(15)
    html = browser.page_source
    time.sleep(1)
    browser.close()
    dfs = pd.read_html(html)
    df_1 = dfs[8]
    df_1 = df_1.drop(df_1.columns[[1,3]], axis = 1)
    df_1 = df_1.drop(0, axis = 0)
    df_2 = dfs[10]
    df_2 = df_2.drop(df_2.columns[[1,3]], axis = 1)
    df_2 = df_2.drop(0, axis = 0)
    df = pd.concat([df_1, df_2])
    df = df.reset_index()
    df["request"] = keyword
    df = df.drop(df.columns[[0]], axis = 1)
    df_list.append(df)
    #display(df)
    #df.to_csv(keyword.replace(" ", "")+'_week.csv' , sep=";",encoding='utf-8-sig')
df_all=pd.concat(df_list)

In [5]:
df_all['date_from']= df_all['Период'].str.split('-', n=1, expand=True)[0].str.strip()
df_all['date_to']= df_all['Период'].str.split('-', n=1, expand=True)[1].str.strip()
df_all.date_from = pd.to_datetime(df_all.date_from,  format = '%d.%m.%Y')
df_all.date_to = pd.to_datetime(df_all.date_to, format = '%d.%m.%Y')

In [6]:
# data from wordstat in df_all
#display(df_all)
df_all = df_all.reset_index()

In [5]:
db = create_engine(conn_string)
conn = db.connect()

In [6]:
print("Reading yt_reels_stat table with latest date...")
df_old =  pd.read_sql("SELECT * FROM wordstat_weekly", conn, parse_dates=['date_from', 'date_to'])
print("Done.")

Reading yt_reels_stat table with latest date...
Done.


In [58]:
df_res = pd.concat([df_old, df_all])

In [65]:
df_res = df_res.drop_duplicates(subset = ['Период', 'request'], keep = 'last') 

In [68]:
#display(df_res)

In [67]:
inject_dict = { 'Период': sqlalchemy.types.TEXT(),  
                'Абсолютное': sqlalchemy.types.BIGINT(), 
                'request': sqlalchemy.types.TEXT(),  
                'date_from': sqlalchemy.types.Date(),                   
                'date_to': sqlalchemy.types.Date(), 
                 }
start_time = time.time()
res = df_res.to_sql('wordstat_weekly', con=conn, if_exists='replace', index=False, dtype = inject_dict)
print("to_sql duration: {} seconds".format(time.time() - start_time))

to_sql duration: 5.560671091079712 seconds
